<div style="width:100%; background-color: #000041"><a target="_blank" href="bit.ly/45U3TvK"><img src="assets/YugabyteDB_DSS-Virtual_LinkedIn-Cover_1584x396.jpg" /></a></div>

<br>

> **Migrate from MySQL to Distributed, Highly Available PostgreSQL!**
>
> Registration is free  [Distributed SQL Summit](bit.ly/45U3TvK).
>

This notebook file is:

`04_Migration_Workflow_Schema_2.ipynb`

## 🛠️ Requirements
Here are the requirements for this notebook:
- ✅ Create the notebook variables in `01_Lab_Setup.ipynb`, which you previously did
- ✅ Create the `ds_ybu` database, which you previously did
- ☑️ Import the notebook variables, *which you must do next*
- ☑️ Analyze, modify, an import the data *which you must do next*


### Select your notebook kernel
- In the Notebook toolbar, click **Select Kernel**.
<br>
<img width=50% src="assets/01_01_Select_Kernel_Toolbar.png" />

- Next, in the dropdown, select **Python 3.8.13** or higher.
<br>
<img width=50% src="assets/01_02_Select_Kernel_Dropdown.png" />

That's it!

## ⛑️ Getting help
The best way to get help us from the Yugabyte University team. You can post your question in YugabyteDB Community Slack in the #training or #yb-university channels. Join our [community](https://communityinviter.com/apps/yugabyte-db/register).

## 👣 Set up steps for this notebook
Here are the steps to setup this lab:

- Import the notebook variables

### 👇 Import the notebook variables

> 👉 IMPORTANT! 👈
> 
> Do **NOT** skip running the following cell. 
> 

The following Python cell reads the stored variables created in the `01_Lab_Setup.ipynb` notebook. To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell. 

👇 👇 👇 

In [ ]:
# Use %store -r to read 01_Lab_Setup variables
%store -r MY_YB_PATH
%store -r MY_YB_PATH_DATA

%store -r MY_GITPOD_WORKSPACE_URL

%store -r MY_DB_NAME
%store -r MY_DB_PORT
%store -r MY_MYSQL_DB_NAME

%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03

%store -r MY_TSERVER_WEBSERVER_PORT
%store -r MY_MYSQL_PORT

%store -r MY_YB_MASTER_HOST_GITPOD_URL
%store -r MY_YB_TSERVER_HOST_GITPOD_URL

%store -r MY_NOTEBOOK_DIR
%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_NOTEBOOK_UTILS_FOLDER
%store -r MY_NOTEBOOK_VOYAGER_TMP

%store -r MY_UTIL_FUNCTIONS_FILE
%store -r MY_UTIL_YBTSERVER_METRICS_FILE

%store -r BETA_FAST_DATA_EXPORT

---
# Migration Workflow: Schema (2)


<div style="width:100%; background-color: #000041"><img src="assets/Migration_Workflow_03.png" /></div>

---
# Import the modified schema

 `yb-voyager` imports the source database into the `public` schema of the target database. This is applicable only for MySQL and Oracle databases. 
 
 By specifying `--target-db-schema` argument during import, you can instruct `yb-voyager` to create a non-public schema and use it for the import.



In [ ]:
%%bash

yb-voyager import schema --help

Import the modified schema to the target YugabyteDB database using the `yb-voyager import schema` command.

In [ ]:
%%bash -s "$MY_DB_NAME"  "$MY_DB_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}

yb-voyager import schema  --start-clean -y    \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --target-db-host ${DB_HOST} \
  --target-db-port  ${DB_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user "ybvoyager" \
  --target-db-password "Password" \
  --target-db-schema "sakila" > /dev/null


We'll, that raised an error: 

`error: ERROR: in an aggregate with DISTINCT, ORDER BY expressions must appear in argument list (SQLSTATE 42P10)`

There may be more than one!

---
## Resolve the schema errors

Oh no! Errors!

Grep the log files to get a better understanding of the related errors. Start with the first one that has to do with a table. Typically resolving errors with the DDL of a table will help solve other errors with views and other dependent objects.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log
 
tail -n 2000 $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG| grep 'SQLSTATE'

`: ERROR: type "geometry" does not exist (SQLSTATE 42704)`
```
2023-09-08 17:52:40 ERROR importData.go:942 DDL Execution Failed for "CREATE TABLE address (\n\taddress_id serial,\n\taddress varchar(50) NOT NULL,\n\taddress2 varchar(50),\n\tdistrict varchar(20) NOT NULL,\n\tcity_id integer NOT NULL,\n\tpostal_code varchar(10),\n\tphone varchar(20) NOT NULL,\n\tlocation GEOMETRY NOT NULL,\n\tlast_update timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,\n\tPRIMARY KEY (address_id)\n) ;": ERROR: type "geometry" does not exist (SQLSTATE 42704)
```

View the table definition from the source database as well as row.

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  # create database
DB_NAME=${1}

mysql -h 0 -u root ${DB_NAME} -e "SHOW CREATE TABLE address; "  
mysql -h 0 -u root ${DB_NAME} -e "select location from address where location is not null limit 1; "  

The GEOMETRY datatype for  MYSQL 8.0+ an be in several formats:
- Well-Known Text (WKT) Format
- Well-Known Binary (WKB) Format
- Internal Geometry Storage Format

> Learn more
> 
> For more details, visit:
> [https://docs.yugabyte.com/preview/api/ysql/datatypes/type_binary/](https://docs.yugabyte.com/preview/api/ysql/datatypes/type_binary/)

Modify the DDL of the `address` table to use a different datatype in the export schema file. Change the data type to a binary data type, `BYTEA`, which is variable length binary string. 

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

SEARCH="GEOMETRY"
REPLACE="BYTEA"
FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/table.sql

sed -i "s/$SEARCH/$REPLACE/" $FILE_PATH

Confirm the change to the file.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/table.sql

cat $FILE_PATH  | grep -B 10 'BYTEA'

Resolve another error. In this case, look at the error for the view, `customer_list`.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log
 
tail -n 2000 $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG| grep 'SQLSTATE'

```
DDL Execution Failed for "CREATE OR REPLACE VIEW customer_list AS select cu.customer_id AS ID,concat(cu.first_name,' ',cu.last_name) AS name,a.address AS address,a.postal_code AS zip code,a.phone AS phone,city.city AS city,country.country AS country,CASE WHEN cu.active THEN 'active' ELSE '' END  AS notes,cu.store_id AS SID FROM (((customer cu join address a on (cu.address_id = a.address_id)) join city on (a.city_id = city.city_id)) join country on (city.country_id = country.country_id));": ERROR: syntax error at or near "code" (SQLSTATE 42601)
```


Use sed or the open editor to make your changes.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

SEARCH="zip code"
REPLACE="zip_code"
FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/views/view.sql

sed -i "s/$SEARCH/$REPLACE/" $FILE_PATH
gp open $FILE_PATH

Verify the change.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/views/view.sql

cat $FILE_PATH  | grep 'zip_code'

---
## Rinse & Repeat: Import the modified schema (1)

In [ ]:
%%bash -s "$MY_DB_NAME"  "$MY_DB_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log

# delete the log so that we can get a fresh view
rm -rf $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG

yb-voyager import schema  --start-clean  -y  \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --target-db-host ${DB_HOST} \
  --target-db-port  ${DB_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user "ybvoyager" \
  --target-db-password "Password" \
  --target-db-schema "sakila" > /dev/null


Another error! 

See the view, `actor_info`.

`ERROR: in an aggregate with DISTINCT, ORDER BY expressions must appear in argument list (SQLSTATE 42P10)`

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log
 
tail -n 2000 $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG| grep 'SQLSTATE'

---
## Rinse & Repeat: Resolve the schema errors (1)

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/views/view.sql

cat $FILE_PATH  | grep -A 1 'CREATE OR REPLACE VIEW actor_info'

The challenge here has to do with the functions, `array_to_string() and array_agg()` . These are being used to replace the `group_concat()` built-in function of MySQL. The best solution, which is long, is to comment out the view, and after successfully importing the schema and data, do the following: 
- create the `_group_concat` function 
- create the `group_concat` aggregate function 
- create the `actor_info` view that uses the `group_concat` function 

It looks something like the following --> don't run this code

In [ ]:
/* do not run this cell */
CREATE FUNCTION _group_concat(text, text) RETURNS text
    AS $_$
SELECT CASE
  WHEN $2 IS NULL THEN $1
  WHEN $1 IS NULL THEN $2
  ELSE $1 || ', ' || $2
END
$_$
    LANGUAGE sql IMMUTABLE;


ALTER FUNCTION sakila._group_concat(text, text) OWNER TO yugabyte;


CREATE AGGREGATE group_concat(text) (
    SFUNC = _group_concat,
    STYPE = text
);


ALTER AGGREGATE sakila.group_concat(text) OWNER TO yugabyte;

CREATE VIEW actor_info AS
    SELECT a.actor_id, 
    a.first_name, 
    a.last_name, 
    group_concat(DISTINCT (((c.name)::text || ': '::text) || (SELECT group_concat((f.title)::text) AS group_concat FROM ((film f JOIN film_category fc ON ((f.film_id = fc.film_id))) JOIN film_actor fa ON ((f.film_id = fa.film_id))) WHERE ((fc.category_id = c.category_id) AND (fa.actor_id = a.actor_id)) GROUP BY fa.actor_id))) AS film_info 
    FROM (((actor a LEFT JOIN film_actor fa ON ((a.actor_id = fa.actor_id))) LEFT JOIN film_category fc ON ((fa.film_id = fc.film_id))) LEFT JOIN category c ON ((fc.category_id = c.category_id))) 
    GROUP BY a.actor_id, a.first_name, a.last_name;

But for now, simply remove the distinct clause from the view.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

SEARCH="distinct"
REPLACE=""
FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/views/view.sql

sed -i "s/$SEARCH/$REPLACE/" $FILE_PATH

---
## Rinse & Repeat: Import the modified schema (2)

In [ ]:
%%bash -s "$MY_DB_NAME"  "$MY_DB_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log

# delete the log so that we can get a fresh view
rm -rf $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG

yb-voyager import schema  --start-clean  -y  \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --target-db-host ${DB_HOST} \
  --target-db-port  ${DB_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user "ybvoyager" \
  --target-db-password "Password" \
  --target-db-schema "sakila" > /dev/null


Another error!

`error: ERROR: argument of CASE/WHEN must be type boolean, not type smallint (SQLSTATE 42804)`

Look at the log for more details.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log
 
tail -n 2000 $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG | grep 'SQLSTATE'

```
 Execution Failed for "CREATE OR REPLACE VIEW customer_list ....
 CASE WHEN cu.active THEN 'active' ELSE '' END 
```

The `cu` alias is the  `customer` table, and the issue is with the `active` column.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/table.sql

cat $FILE_PATH  | grep -A 11 'CREATE TABLE customer'

The `active` column uses `smallint`. 

---
## Rinse & Repeat: Resolve the schema errors (2)

Update the `table.sql` file using sed or the editor. 

Change the `smallint` to a `boolean`.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

SEARCH="active smallint NOT NULL DEFAULT 1"
REPLACE="active boolean NOT NULL DEFAULT TRUE"
FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/table.sql

sed -i "s/$SEARCH/$REPLACE/" $FILE_PATH
gp open $FILE_PATH


Verify the changes

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/tables/table.sql

cat $FILE_PATH  | grep -A 11 'CREATE TABLE customer'

---
## Rinse & Repeat: Import the modified schema (3)

Rerun `import schema` again.

In [ ]:
%%bash -s "$MY_DB_NAME"  "$MY_DB_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log

# delete the log so that we can get a fresh view
rm -rf $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG

yb-voyager import schema  --start-clean  -y  \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --target-db-host ${DB_HOST} \
  --target-db-port  ${DB_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user "ybvoyager" \
  --target-db-password "Password" \
  --target-db-schema "sakila" > /dev/null

Oops! Another error:

`error: ERROR: column "c.city" must appear in the GROUP BY clause or be used in an aggregate function (SQLSTATE 42803)`

This error has to do the view, `sales_by_store`:

```
DDL Execution Failed for "CREATE OR REPLACE VIEW sales_by_store AS select concat(c.city,',',cy.country) AS store,concat(m.first_name,' ',m.last_name) AS manager,sum(p.amount) AS total_sales FROM (((((((payment p join rental r on (p.rental_id = r.rental_id)) join inventory i on (r.inventory_id = i.inventory_id)) join store s on (i.store_id = s.store_id)) join address a on (s.address_id = a.address_id)) join city c on (a.city_id = c.city_id)) join country cy on (c.country_id = cy.country_id)) join staff m on (s.manager_staff_id = m.staff_id)) group by s.store_id order by cy.country,c.city;": ERROR: column "c.city" must appear in the GROUP BY clause or be used in an aggregate function (SQLSTATE 42803)
```

---
## Rinse & Repeat: Resolve the schema errors (3)

PostgreSQL, like most sensible DBMSs, does "group by" properly, insisting that everything that you select must be either "grouped by" or be aggregated through a function, like sum(). 

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

FILE_PATH=$NOTEBOOK_VOYAGER_TMP/export-dir/schema/views/view.sql

gp open $FILE_PATH

Use the editor to replace the view with the following:

In [ ]:
CREATE OR REPLACE VIEW sales_by_store AS 
select 
    concat(c.city,',',cy.country) AS store,
    concat(m.first_name,' ',m.last_name) AS manager,
    sum(p.amount) AS total_sales 
FROM (((((((payment p join rental r on (p.rental_id = r.rental_id)) join inventory i on (r.inventory_id = i.inventory_id)) join store s on (i.store_id = s.store_id)) join address a on (s.address_id = a.address_id)) join city c on (a.city_id = c.city_id)) join country cy on (c.country_id = cy.country_id)) join staff m on (s.manager_staff_id = m.staff_id))
group by c.city, cy.country, m.first_name, m.last_name
order by cy.country, c.city;

Save your changes.

---
## Rinse & Repeat: Import the modified schema (4)

Rerun `import schema` again.

In [ ]:
%%bash -s "$MY_DB_NAME"  "$MY_DB_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}
IMPORT_SCHEMA_LOG=yb-voyager-import-schema.log

# delete the log so that we can get a fresh view
rm -rf $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$IMPORT_SCHEMA_LOG

yb-voyager import schema  --start-clean  -y  \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --target-db-host ${DB_HOST} \
  --target-db-port  ${DB_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user "ybvoyager" \
  --target-db-password "Password" \
  --target-db-schema "sakila" > /dev/null
  

Great! That worked!
No errors! Nicely done 🙌 

---
## Sanity check: Verify the imported schema in YugabyteDB

First, list the schemas using the `\dn` meta command.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # create database
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

# meta commands

./bin/ysqlsh -d ${DB_NAME} -U ybvoyager -c "\dn"

Next, list the tables using the `\dt` meta command.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # create database
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

# meta commands

./bin/ysqlsh -d ${DB_NAME} -U ybvoyager -c "\dt sakila."


Verify the views using the `\dv` meta command.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # create database
YB_PATH=${1}
DB_NAME=${2}

cd $YB_PATH

./bin/ysqlsh -d ${DB_NAME} -U ybvoyager -c "\dv sakila."

Verify the index creation with a PostgreSQL query.


In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # create database
YB_PATH=${1}
DB_NAME=${2}
SQL_PG_INDEXES="select tablename, indexname, indexdef from pg_indexes where schemaname = 'sakila' order by tablename, indexname;"
cd $YB_PATH

# query

./bin/ysqlsh -d ${DB_NAME} -U ybvoyager -c "${SQL_PG_INDEXES}"

Verify the trigger creation with a PostgreSQL query.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"  # create database
YB_PATH=${1}
DB_NAME=${2}

SQL_PG_TRIGGERS="select  event_object_table AS table_name, trigger_name from information_schema.triggers where trigger_schema = 'sakila' group by table_name, trigger_name order by table_name, trigger_name;"


cd $YB_PATH
./bin/ysqlsh -d ${DB_NAME} -U ybvoyager -c "${SQL_PG_TRIGGERS}"

---
## Export source data
Export the source data from the source database using `yb-voyager`.  First, review the help for the command, `yb-voyager export data`.


In [ ]:
%%bash

yb-voyager export data --help


> Note:
> 
> The `source-db-schema` argument is not applicable to MySQL.
>
> The `-y` argument says yes to all prompts.
>
> BETA_FAST_DATA_EXPORT=1 is an environment variable.
> 

Run the export data command.

In [ ]:
%%bash -s "$MY_MYSQL_DB_NAME"  "$MY_MYSQL_PORT" "$MY_HOST_IPv4_01" "$MY_NOTEBOOK_VOYAGER_TMP"
DB_NAME=${1}
DB_PORT=${2}
DB_HOST=${3}
NOTEBOOK_VOYAGER_TMP=${4}

yb-voyager export data -y --start-clean --disable-pb --send-diagnostics false \
  --export-dir $NOTEBOOK_VOYAGER_TMP/export-dir \
  --source-db-type "MYSQL" \
  --source-db-host ${DB_HOST} \
  --source-db-port  ${DB_PORT} \
  --source-db-name  ${DB_NAME} \
  --source-db-user "ybvoyager" \
  --source-db-password "Yugabyte#1" > /dev/null
  
       

> Problems?
> 
> In case there are issues with the above and you need to kill the process, you can uncomment and run the following:

In [ ]:
%%bash 
# ps -eaf | grep yb-voyager | grep -v grep | awk '{ print $2 }' | xargs kill -9
# ps -aux | grep 'yb-voyager'

Review the Voyager log file for the export.

In [ ]:
%%bash -s "$MY_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}
EXPORT_DATA_LOG=yb-voyager-export-data.log

tail -n 200 $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$EXPORT_DATA_LOG
gp open $NOTEBOOK_VOYAGER_TMP/export-dir/logs/$EXPORT_DATA_LOG

---

## 😊 Next up!
Continue your learning by opening the next notebook, `05_Migration_Workflow_Import.ipynb`. 

Or, if using GitPod, run the following cell:

In [ ]:
%%bash
gp open '05_Migration_Workflow_Import.ipynb'